In [1]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np
import scipy.io
from skimage.transform import downscale_local_mean
import os
import sys
import json
import math
import time
import random
from random import shuffle
import pickle
import h5py
import glob
from scipy import stats

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SequentialSampler
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
import numpy as np

cuda:0


In [3]:
#useful for debugging
#whenever we want to print something
torch.set_printoptions(threshold=1000)

In [4]:
#set seed
random.seed(0)
np.random.seed(0)

#path for data set
dataset_paths = ['dataset/UCF_CC_50']

slice_w = 256
slice_h = 256

patch_w = 225
patch_h = 225

net_density_h = 28
net_density_w = 28

HAS_GPU = True
GPU_ID = 0

In [5]:
#mean
VGG_ILSVRC_16_layers_mean = np.zeros((3, patch_h, patch_w), dtype='f4')
VGG_ILSVRC_16_layers_mean[0,:,:] = 103.939
VGG_ILSVRC_16_layers_mean[1,:,:] = 116.779
VGG_ILSVRC_16_layers_mean[2,:,:] = 123.68

In [6]:
def load_gt_from_json(gt_file, gt_shape):
    #numpy array for ground truth 
    gt = np.zeros(gt_shape, dtype='uint8') 
    #open the file to read
    with open(gt_file, 'r') as jf:
        for j, dot in enumerate(json.load(jf)):
            try:
                #for row y
                #for column x
                #head annotation
                gt[int(math.floor(dot['y'])), int(math.floor(dot['x']))] = 1
            except IndexError:
                print(gt_file, dot['y'], dot['x'], sys.exc_info())
    return gt

def load_images_and_gts(path):
    images = []
    gts = []
    densities = []
    for gt_file in glob.glob(os.path.join(path, '*.json')):
        print(gt_file)
        #read input image
        if os.path.isfile(gt_file.replace('.json','.png')):
            img = cv2.imread(gt_file.replace('.json','.png'))
        else:
            img = cv2.imread(gt_file.replace('.json','.jpg'))
        images.append(img)
        
        #load corrsponding ground truth
        gt = load_gt_from_json(gt_file, img.shape[:-1])
        gts.append(gt)
        
        #densities
        desnity_file = gt_file.replace('.json','.h5')
        if os.path.isfile(desnity_file):
            #load density if exist
            with h5py.File(desnity_file, 'r') as hf:
                density = np.array(hf.get('density'))
        else:
            density = gaussian_filter_density([gt])[0]
            with h5py.File(desnity_file, 'w') as hf:
                hf['density'] = density
        densities.append(density)
    print(path, len(images), 'loaded')
    return (images, gts, densities)

def density_resize(density, fx, fy):
    return cv2.resize(density, None, fx=fx, fy=fy, interpolation = cv2.INTER_CUBIC)/(fx*fy)

#this function just scales images 
#resizes given set of images
#and returns list of all images
def multiscale_pyramidal(images, gts, start=0.5, end=1.3, step=0.1):
    frange = np.arange(start, end, step)
    out_images = []
    out_gts = []
    for i, img in enumerate(images):
        for f in frange:
            out_images.append(cv2.resize(img, None, fx=f, fy=f, interpolation = cv2.INTER_CUBIC))
            out_gts.append(density_resize(gts[i], fx=f, fy=f))
    return (out_images, out_gts)

def adapt_images_and_densities(images, gts, slice_w=slice_w, slice_h=slice_h):
    out_images = []
    out_gts = []
    for i, img in enumerate(images):
        img_h, img_w, _ = img.shape
        n_slices_h = int(round(img_h/slice_h))
        n_slices_w = int(round(img_w/slice_w))
        new_img_h = float(n_slices_h * slice_h)
        new_img_w = float(n_slices_w * slice_w)
        fx = new_img_w/img_w
        fy = new_img_h/img_h
        out_images.append(cv2.resize(img, None, fx=fx, fy=fy, interpolation = cv2.INTER_CUBIC))
        assert out_images[-1].shape[0]%slice_h == 0 and out_images[-1].shape[1]%slice_w == 0
        if gts is not None:
            out_gts.append(density_resize(gts[i], fx, fy))
    return (out_images, out_gts)

#Generate slices
#like sliding window
def generate_slices(images, gts, slice_w=slice_w, slice_h=slice_h, offset=None):
    if offset == None:
        offset = slice_w
    out_images = []
    out_gts = []
    #iterate though images
    for i, img in enumerate(images):
        img_h, img_w, _ = img.shape        
        p_y_id = 0
        p_y1 = 0
        p_y2 = p_y1 + slice_h
        #slide vertically
        while p_y2 <= img_h:
            p_x_id = 0
            p_x1 = 0
            p_x2 = p_x1 + slice_w
            #slide horizontally
            while p_x2 <= img_w:
                out_images.append(img[p_y1:p_y2,p_x1:p_x2])
                assert out_images[-1].shape[:-1] == (slice_h, slice_w)
                if gts is not None:
                    out_gts.append(gts[i][p_y1:p_y2,p_x1:p_x2])
                    assert out_gts[-1].shape == (slice_h, slice_w)
                #next
                p_x_id += 1
                p_x1 += offset
                p_x2 += offset
            p_y_id += 1
            p_y1 += offset
            p_y2 += offset
    return (out_images, out_gts)

#Data augmentation: CROP
def crop_slices(images, gts):
    out_images = []
    out_gts = []
    for i, img in enumerate(images):
        #data augmentation
        #crop-5
        img_h, img_w, _ = img.shape
        gt = gts[i]
        #top-left
        p_y1, p_y2 = 0, patch_h
        p_x1, p_x2 = 0, patch_w
        out_images.append(img[p_y1:p_y2,p_x1:p_x2])
        out_gts.append(gt[p_y1:p_y2,p_x1:p_x2])
        #top-right
        p_y1, p_y2 = 0, patch_h
        p_x1, p_x2 = img_w-patch_w, img_w
        out_images.append(img[p_y1:p_y2,p_x1:p_x2])
        out_gts.append(gt[p_y1:p_y2,p_x1:p_x2])
        #bottom-left
        p_y1, p_y2 = img_h-patch_h, img_h
        p_x1, p_x2 = 0, patch_w
        out_images.append(img[p_y1:p_y2,p_x1:p_x2])
        out_gts.append(gt[p_y1:p_y2,p_x1:p_x2])
        #bottom-right
        p_y1, p_y2 = img_h-patch_h, img_h
        p_x1, p_x2 = img_w-patch_w, img_w
        out_images.append(img[p_y1:p_y2,p_x1:p_x2])
        out_gts.append(gt[p_y1:p_y2,p_x1:p_x2])
        #center
        p_y1, p_y2 = int((img_h-patch_h)/2), int((img_h-patch_h)/2)+patch_h
        p_x1, p_x2 = int((img_w-patch_w)/2), int((img_w-patch_w)/2)+patch_w
        out_images.append(img[p_y1:p_y2,p_x1:p_x2])
        out_gts.append(gt[p_y1:p_y2,p_x1:p_x2])
    return (out_images, out_gts)


#Data augmentation: FLIP
def flip_slices(images, gts):
    out_images = []
    out_gts = []
    for i, img in enumerate(images):
        img_h, img_w, _ = img.shape
        gt = gts[i]
        #original images
        out_images.append(img)
        out_gts.append(gt)
        #flip: left-right
        out_images.append(np.fliplr(img))
        out_gts.append(np.fliplr(gt))
    return (out_images, out_gts)

#Shuffling
def shuffle_slices(images, gts):
    out_images = []
    out_gts = []
    index_shuf = list(range(len(images)))
    shuffle(index_shuf)
    for i in index_shuf:
        out_images.append(images[i])
        out_gts.append(gts[i])
    return (out_images, out_gts)

def samples_distribution(images, gts):
    out_images = []
    out_gts = []
    gts_count = list(map(np.sum, gts))
    max_count = max(gts_count)
    #pos
    for i, img in enumerate(images):
        if gts_count[i] >= 1. and random.random() < gts_count[i]**2/max_count**2:
            out_images.append(img)
            out_gts.append(gts[i])
    #neg
    neg_count = sum(gt_count < 1. for gt_count in gts_count)
    obj_neg_count = len(out_gts) / 6 # ~= 15-16%
    neg_keep_prob = min(1., float(obj_neg_count) / float(neg_count))
    for i, img in enumerate(images):
        if gts_count[i] < 1. and random.random() < neg_keep_prob:
            out_images.append(img)
            out_gts.append(gts[i])
        
    return (out_images, out_gts)

#to generate densities as a label
def gaussian_filter_density(gts):
    densities = []
    for gt in gts:
        print(gt.shape)
        density = np.zeros(gt.shape, dtype=np.float32)
        gt_count = np.count_nonzero(gt)
        if gt_count == 0:
            return density

        pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
        leafsize = 2048
        # build kdtree
        #print 'build kdtree...'
        tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
        # query kdtree
        #print 'query kdtree...' 
        distances, locations = tree.query(pts, k=2, eps=10.)

        #print 'generate density...'
        
        for i, pt in enumerate(pts):
            pt2d = np.zeros(gt.shape, dtype=np.float32)
            pt2d[pt[1],pt[0]] = 1.
            if gt_count > 1:
                sigma = distances[i][1]
            else:
                sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
            density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
        #print 'done.'
        densities.append(density)
    return densities

In [7]:
# Positive image and ground truth loading
X_fs = []
Y_fs = []

for path in dataset_paths:
    images, gts, densities = load_images_and_gts(path)
    X_fs += images
    Y_fs += densities

dataset/UCF_CC_50/16.json
dataset/UCF_CC_50/3.json
dataset/UCF_CC_50/45.json
dataset/UCF_CC_50/18.json
dataset/UCF_CC_50/4.json
dataset/UCF_CC_50/41.json
dataset/UCF_CC_50/23.json
dataset/UCF_CC_50/39.json
dataset/UCF_CC_50/21.json
dataset/UCF_CC_50/24.json
dataset/UCF_CC_50/28.json
dataset/UCF_CC_50/27.json
dataset/UCF_CC_50/43.json
dataset/UCF_CC_50/38.json
dataset/UCF_CC_50/5.json
dataset/UCF_CC_50/22.json
dataset/UCF_CC_50/48.json
dataset/UCF_CC_50/6.json
dataset/UCF_CC_50/36.json
dataset/UCF_CC_50/25.json
dataset/UCF_CC_50/9.json
dataset/UCF_CC_50/34.json
dataset/UCF_CC_50/8.json
dataset/UCF_CC_50/35.json
dataset/UCF_CC_50/49.json
dataset/UCF_CC_50/30.json
dataset/UCF_CC_50/37.json
dataset/UCF_CC_50/44.json
dataset/UCF_CC_50/26.json
dataset/UCF_CC_50/10.json
dataset/UCF_CC_50/42.json
dataset/UCF_CC_50/46.json
dataset/UCF_CC_50/29.json
dataset/UCF_CC_50/33.json
dataset/UCF_CC_50/31.json
dataset/UCF_CC_50/19.json
dataset/UCF_CC_50/11.json
dataset/UCF_CC_50/47.json
dataset/UCF_CC_50/

In [8]:
for i in Y_fs:
    print(i.shape)

(600, 800)
(752, 1024)
(600, 896)
(768, 1024)
(680, 1024)
(656, 1024)
(656, 1024)
(680, 1024)
(1024, 680)
(648, 1024)
(680, 1024)
(432, 1024)
(496, 744)
(328, 496)
(424, 640)
(680, 1024)
(768, 1024)
(768, 1024)
(768, 1024)
(712, 1024)
(768, 1024)
(768, 1024)
(768, 1024)
(768, 1024)
(1024, 1024)
(480, 640)
(768, 1024)
(680, 1024)
(712, 1024)
(448, 640)
(600, 456)
(656, 1024)
(680, 1024)
(424, 720)
(680, 1024)
(496, 360)
(744, 1024)
(600, 800)
(680, 1024)
(440, 640)
(768, 1024)
(680, 1024)
(680, 1024)
(824, 1024)
(424, 600)
(528, 640)
(680, 1024)


In [9]:
# Split test an train set
from sklearn.model_selection import train_test_split

# Train 80%, Test 20%
X_fs_train, X_fs_test, Y_fs_train, Y_fs_test = train_test_split(X_fs, Y_fs, test_size=0.2)

# FS: FULL SIZE
X_train, Y_train = X_fs_train, Y_fs_train
X_test, Y_test = X_fs_test, Y_fs_test

# DATA AUGMENTATION
print('\nMultiscale pyramidal')
print('TRAIN:')
X_train, Y_train = multiscale_pyramidal(X_train, Y_train)
print(len(X_train), len(Y_train))
print('TEST:')
X_test, Y_test = multiscale_pyramidal(X_test, Y_test)
print(len(X_test), len(Y_test))

# PATCH SIZE
print('\nGenerate slices')
print('TRAIN:')
X_train, Y_train = generate_slices(X_train, Y_train, slice_w=patch_w, slice_h=patch_h, offset=8)
print(len(X_train), len(Y_train))
print('TEST:')
X_test, Y_test = generate_slices(X_test, Y_test, slice_w=patch_w, slice_h=patch_h)
print(len(X_test), len(Y_test))

print('\nFlip')
print('TRAIN:')
X_train, Y_train = flip_slices(X_train, Y_train)
print(len(X_train), len(Y_train))
print('TEST:')
X_test, Y_test = flip_slices(X_test, Y_test)
print(len(X_test), len(Y_test))

print('\nSamples gt distribution correction')
print('TRAIN:')
X_train, Y_train = samples_distribution(X_train, Y_train)
print(len(X_train), len(Y_train))


print('\nShuffle')
print('TRAIN:')
X_train, Y_train = shuffle_slices(X_train, Y_train)
print(len(X_train), len(Y_train))
print('TEST:')
X_test, y_test = shuffle_slices(X_test, Y_test)
print(len(X_test), len(Y_test))


Multiscale pyramidal
TRAIN:
296 296
TEST:
80 80

Generate slices
TRAIN:
1066861 1066861
TEST:
559 559

Flip
TRAIN:
2133722 2133722
TEST:
1118 1118

Samples gt distribution correction
TRAIN:
33006 33006

Shuffle
TRAIN:
33006 33006
TEST:
1118 1118


In [10]:
X_train = np.asarray(X_train)
#convert to float
# X_train = np.float32(X_train)
# X_train = X_train/255.0

X_train = X_train.transpose(0, 3, 1, 2)
X_train_tensor = torch.from_numpy(X_train)

Y_train = np.asarray(Y_train)
Y_train = np.reshape(Y_train,(Y_train.shape[0],1,Y_train.shape[1],Y_train.shape[2]))
Y_train_tensor = torch.from_numpy(Y_train)

In [11]:
X_test = np.asarray(X_test)
X_test = X_test.transpose(0, 3, 1, 2)
X_test_tensor = torch.from_numpy(X_test)

Y_test = np.asarray(Y_test)
Y_test = np.reshape(Y_test,(Y_test.shape[0],1,Y_test.shape[1],Y_test.shape[2]))
Y_test_tensor = torch.from_numpy(Y_test)

In [12]:
#our module should subclass this class
#every network we make should be inherited by "nn.Module"
class CrowdNet(nn.Module):
    def __init__(self):
        super(CrowdNet, self).__init__()
        # 3 image input channel, 64 filters, 3x3 kernel
        # stack them sequentially
        self.deepConvNet = nn.Sequential(
            # c1
            nn.Conv2d(3, 64, kernel_size=(3, 3),padding=1),
            # relu1
            nn.ReLU(),  
            
            
            # c2
            nn.Conv2d(64, 64, kernel_size=(3, 3),padding=1), 
            # relu2
            nn.ReLU(),  
            
            # first pooling layer
            # s1
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),  
            
            # c3
            nn.Conv2d(64, 128, kernel_size=(3, 3),padding=1),
            # relu3
            nn.ReLU(),  
            
            # c4
            nn.Conv2d(128, 128, kernel_size=(3, 3),padding=1),
            # relu4s
            nn.ReLU(),  
            
            # second pooling layer
            # s2
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # c5
            nn.Conv2d(128, 256, kernel_size=(3, 3),padding=1),
            # relu5
            nn.ReLU(),
            
            # c6
            nn.Conv2d(256, 256, kernel_size=(3, 3),padding=1),
            # relu6
            nn.ReLU(),
            
            # c7
            nn.Conv2d(256, 256, kernel_size=(3, 3),padding=1),
            # relu7
            nn.ReLU(),
            
            # third pooling layer
            # s3
            nn.MaxPool2d(kernel_size=(2, 2), stride=2),
            
            # c8
            nn.Conv2d(256, 512, kernel_size=(3, 3),padding=1),
            # relu8
            nn.ReLU(),
            
            # c9
            nn.Conv2d(512, 512, kernel_size=(3, 3),padding=1),
            # relu9
            nn.ReLU(),
            
            # c10
            nn.Conv2d(512, 512, kernel_size=(3, 3),padding=1),
            # relu10
            nn.ReLU(),
            
            # fourth pooling layer
            # s4
            nn.MaxPool2d(kernel_size=(3, 3), stride=1),
            
            # c11
            nn.Conv2d(512, 512, kernel_size=(3, 3),padding=1),
            # relu11
            nn.ReLU(),
            
            # c12
            nn.Conv2d(512, 512, kernel_size=(3, 3),padding=1),
            # relu12
            nn.ReLU(),
            
            # c13
            nn.Conv2d(512, 512, kernel_size=(3, 3),padding=1),
            # relu13
            nn.ReLU()
        )
        
        self.shallowConvNet = nn.Sequential(
            # c1
            nn.Conv2d(3, 24, kernel_size=(5, 5),padding=2),
            # relu1
            nn.ReLU(),
            
            #average pooling layer 1
            nn.AvgPool2d(kernel_size=(5, 5), stride=2),
            
            # c2
            nn.Conv2d(24, 24, kernel_size=(5, 5),padding=2),
            # relu2
            nn.ReLU(),
            
            #average pooling layer 2
            nn.AvgPool2d(kernel_size=(5, 5), stride=2),
            
            # c3
            nn.Conv2d(24, 24, kernel_size=(5, 5),padding=3),
            # relu3
            nn.ReLU(),
            
            
            #average pooling layer 3
            nn.AvgPool2d(kernel_size=(5, 5), stride=2)
        )
        
        self.interpConvNet = nn.Sequential(
            # c1
            nn.Conv2d(536, 1, kernel_size=(1, 1)),
        )
        
    def forward(self, input):
        deepConvOut = self.deepConvNet(input)
        shallowConvOut = self.shallowConvNet(input)
        #concatanate two layers
        concOut = torch.cat((deepConvOut, shallowConvOut), 1)
        
        interpOut = self.interpConvNet(concOut)
        
        inputW = input.size()[2]
        inputH = input.size()[3]
        
        #scale up the image 
        scaleUp = nn.Upsample(size=(inputW,inputH), mode='bilinear',align_corners = True)
        scaledInterpOut = scaleUp(interpOut)
        return scaledInterpOut

In [13]:
# learning_rate = 2e-3

# model = CrowdNet().to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# a = np.zeros((496,744,3),dtype='float32')
# print(a.shape)

# model.eval()

# a = torch.from_numpy(a)
# a = a.to(device)
# a = a.view(1,a.size()[2],a.size()[0],a.size()[1])
# outputs = model(a)
# print(outputs.size())

In [14]:
#define model related constants
num_epochs = 30
batch_size_train = 16
batch_size_test = 16
learning_rate = 1e-4

model = CrowdNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)

In [15]:
print(torch.cuda.memory_allocated(device))

59905024


In [16]:
import torch.utils.data as utils

#loaders for mini batch
crowd_count_dataset = utils.TensorDataset(X_train_tensor,Y_train_tensor) # create your datset
crowd_count_trainloader = utils.DataLoader(crowd_count_dataset, batch_size=batch_size_train, shuffle=False) # create your dataloader

crowd_count_dataset_test = utils.TensorDataset(X_test_tensor,Y_test_tensor) # create your datset
crowd_count_testloader = utils.DataLoader(crowd_count_dataset_test, batch_size=batch_size_test, shuffle=False) # create your dataloader

In [17]:
dtype = torch.FloatTensor

if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor

In [18]:
def train_one_epoch(epoch_num):
    for i, (images, labels) in enumerate(crowd_count_trainloader):
        images = images.to(device).type(dtype)
#         images = images/255.0
        labels = labels.to(device).type(dtype)
        # Forward pass
        optimizer.zero_grad() 
        outputs = model(images) 
        loss = criterion(outputs, labels)
        # Backward pass
        loss.backward()
        # Optimize
        optimizer.step()
        if (i + 1) % 10 == 0:
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.8f'
                  % (epoch_num + 1, num_epochs, i + 1,
                     len(crowd_count_dataset) // batch_size_train, loss.item()))
#         print(torch.cuda.memory_allocated(device))

In [19]:
def epoch_error(loader, length, split='validation'):
    """ Computes the loss for all data points in a loader.
       
        Inputs:
            loader: Pytorch data loader (object)
            length: Number of data points (integer)
            split: Name of split, typically 'train', 'test', or 'validation' (string)
        
        Returns:
            loss (floating point)
    """
    model.eval()
    # Measure the error for the entire loader split.
    i = 0
    loss = 0.
    num_batches = 0
    for images, labels in loader:  # One batch at a time!
        images = images.to(device).type(dtype)
#         images = images/255.0
        labels = labels.to(device).type(dtype)
        outputs = model(images)
        loss += criterion(outputs, labels).item()
        # print(f'Batch loss is {criterion(outputs, labels)}')
        num_batches += 1

    print(f'Error of the model on the {length} {split} images: {loss / num_batches: .6f}')
    return loss / num_batches

In [20]:
trainError = []
testError = []
def run_training():
    for epoch in range(num_epochs):
        train_one_epoch(epoch)
        #store the model also
        modelStr = "./LastSavedDSParam_%d.pt"%epoch
        torch.save(model.state_dict(), modelStr)
        temp = epoch_error(crowd_count_trainloader, len(crowd_count_dataset), 'train')
        trainError.append(temp)
        temp = epoch_error(crowd_count_testloader, len(crowd_count_dataset_test), 'test')
        testError.append(temp)

In [21]:
print(torch.cuda.memory_allocated(device))

59905024


In [ ]:
run_training()

Epoch: [ 1/ 30], Step: [ 10/ 2062], Loss: 0.02440552
Epoch: [ 1/ 30], Step: [ 20/ 2062], Loss: 0.00198729
Epoch: [ 1/ 30], Step: [ 30/ 2062], Loss: 0.00392830
Epoch: [ 1/ 30], Step: [ 40/ 2062], Loss: 0.00190447
Epoch: [ 1/ 30], Step: [ 50/ 2062], Loss: 0.00109336
Epoch: [ 1/ 30], Step: [ 60/ 2062], Loss: 0.00105917
Epoch: [ 1/ 30], Step: [ 70/ 2062], Loss: 0.00070538
Epoch: [ 1/ 30], Step: [ 80/ 2062], Loss: 0.00076102
Epoch: [ 1/ 30], Step: [ 90/ 2062], Loss: 0.00068463
Epoch: [ 1/ 30], Step: [ 100/ 2062], Loss: 0.00105097
Epoch: [ 1/ 30], Step: [ 110/ 2062], Loss: 0.00072174
Epoch: [ 1/ 30], Step: [ 120/ 2062], Loss: 0.00052875
Epoch: [ 1/ 30], Step: [ 130/ 2062], Loss: 0.00048468
Epoch: [ 1/ 30], Step: [ 140/ 2062], Loss: 0.00071840


In [ ]:
plt.plot(trainError)

In [ ]:
plt.plot(testError)

In [ ]:
for i in range(len(Y_fs)):
    print(Y_fs[i].shape)
    print(np.sum(Y_fs[i]))

In [ ]:
'''
model.eval()
for images in X_fs:
    temp = torch.from_numpy(images)
    temp = temp.to(device).type(dtype)
    temp = temp.view(1,temp.size()[2],temp.size()[0],temp.size()[1])
    outputs = model(temp)
    print(outputs.size())
#     print(torch.sum(outputs))
'''